In [1]:
#import section

import sys
sys.path.append('modules')

import json
import time
import imaplib
import email
from bs4 import BeautifulSoup
import dbstruct
import messagestruct
import messagefuncs
import dbfuncs


In [2]:
#initialisation


with open('gmail.creds') as f:
    data = json.load(f)
    gmailServer = "imap.gmail.com"
    gmailPort   = 993    
    gmailAccount = data['account']
    gmailPassword = data['password']
    
    
with open('db.creds') as f:
    data = json.load(f)
    dbHost = data['host']
    dbPort = data['port']
    dbDatabase = data['database']
    dbUser = data['user']
    dbPassword = data['password'] 

In [3]:
#connect to gmail

mail = imaplib.IMAP4_SSL(gmailServer)
mail.login(gmailAccount,gmailPassword)

('OK', [b'fitosystems@gmail.com authenticated (Success)'])

In [4]:
#querying and defining messages for processing


shiftFromLastMessage = 9

mail.select('fitosystems.ru')
#typ, data = mail.search(None, 'FROM "info@fitosystems.ru"')
typ, data = mail.uid('search', None, 'FROM "info@fitosystems.ru"')


mail_uids = data[0]
uid_list = list(map(lambda x: x.decode("utf-8"), mail_uids.split()))


# first_email_uid = uid_list[0]
# last_email_uid = uid_list[-1 - shiftFromLastMessage]



# try:
#     indexFrom = uid_list.index(dblast_email_uid) + 1
# except ValueError:
#     indexFrom = 0
    
# indexTo = len(uid_list) - 1 - shiftFromLastMessage



In [5]:
#processing instances



engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)
session = dbfuncs.dbGetSession(engine)



 
dbresult = session.query(dbstruct.Order.message_id).all() 
dbuid_list = [str(dbres[0]) for dbres in dbresult]


substractList = list(set(uid_list) - set(dbuid_list))


print("total in gmail - " + str(len(uid_list)))
print("total in db - " + str(len(dbuid_list)))
print("total msgs to process - " + str(len(substractList)))


engine.dispose()

total in gmail - 11402
total in db - 4790
total msgs to process - 6612


In [6]:


engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)
session = dbfuncs.dbGetSession(engine)


cnt = 0
for messageId in substractList:

    typ, data = mail.uid('fetch',str(messageId),'(RFC822)') 
    rawEmail = data[0][1]
    message = email.message_from_bytes(rawEmail)
    
    
    try:
        messageDict = messagefuncs.parseMessage(messageId,message)

        #print(str(messageDict.order['messageId']) + ' ' + message["Date"])
        print(str(cnt) + " OK")

        dbfuncs.submitMessage(session,messageDict)
        
    except BaseException as ex:    
        print(str(cnt) + " !!!error - " + str(ex) + ' ' + message["Date"])
        continue


    if (cnt % 250 == 0):
        session.commit()
        print("commit")
    cnt += 1


try:
    session.commit()
    print("commit")
except BaseException as ex:
    session.rollback()
    print("rollback" + str(ex))
    

engine.dispose()

0 OK
commit
1 OK
2 OK
3 OK
4 OK
5 OK
6 OK
7 OK
8 OK
9 OK
10 OK
11 OK
12 OK
13 OK
14 OK
15 OK
16 OK
17 OK
18 OK
19 !!!error - list index out of range Mon, 13 Nov 2017 16:54:17 +0300
19 OK
20 OK
21 OK
22 OK
23 !!!error - list index out of range Thu, 9 Feb 2017 17:32:02 +0300
23 OK
24 OK
25 OK
26 OK
27 OK
28 OK
29 OK
30 OK
31 OK
32 OK
33 OK
34 OK
35 OK
36 OK
37 OK
38 OK
39 !!!error - invalid literal for int() with base 10: '№' Wed, 4 Mar 2015 14:01:16 +0300
39 OK
40 OK
41 !!!error - list index out of range Sun, 25 Mar 2018 18:35:27 +0300
41 OK
42 OK
43 OK
44 !!!error - list index out of range Fri, 1 Dec 2017 11:30:12 +0300
44 OK
45 !!!error - invalid literal for int() with base 10: 'HA:' Wed, 19 Aug 2015 14:39:48 +0300
45 OK
46 OK
47 !!!error - list index out of range Fri, 25 Aug 2017 14:31:47 +0300
47 OK
48 OK
49 OK
50 OK
51 OK
52 OK
53 OK
54 OK
55 OK
56 OK
57 OK
58 !!!error - list index out of range Wed, 6 Dec 2017 18:24:03 +0300
58 OK
59 OK
60 OK
61 OK
62 OK
63 OK
64 OK
65 OK
66 OK
67 

439 OK
440 OK
441 OK
442 OK
443 !!!error - invalid literal for int() with base 10: '№' Fri, 27 Feb 2015 16:31:32 +0300
443 OK
444 OK
445 OK
446 OK
447 OK
448 OK
449 OK
450 OK
451 OK
452 OK
453 OK
454 OK
455 OK
456 OK
457 !!!error - list index out of range Thu, 23 Nov 2017 12:25:53 +0300
457 OK
458 OK
459 OK
460 OK
461 OK
462 OK
463 OK
464 OK
465 OK
466 OK
467 OK
468 OK
469 OK
470 OK
471 OK
472 OK
473 OK
474 OK
475 OK
476 OK
477 OK
478 OK
479 OK
480 !!!error - list index out of range Thu, 23 Nov 2017 20:27:18 +0300
480 OK
481 OK
482 OK
483 !!!error - invalid literal for int() with base 10: '№' Mon, 16 Feb 2015 13:48:19 +0300
483 OK
484 !!!error - list index out of range Sun, 19 Mar 2017 15:54:50 +0300
484 OK
485 !!!error - list index out of range Thu, 3 Mar 2016 16:42:30 +0300
485 OK
486 OK
487 OK
488 OK
489 OK
490 OK
491 OK
492 OK
493 OK
494 OK
495 !!!error - 'utf-8' codec can't decode byte 0xb0 in position 28: invalid start byte Sun, 01 Jul 2018 02:49:11 +0300
495 OK
496 OK
497 OK
498

873 OK
874 !!!error - list index out of range Fri, 1 Dec 2017 17:34:38 +0300
874 OK
875 OK
876 OK
877 OK
878 OK
879 OK
880 !!!error - invalid literal for int() with base 10: '№' Thu, 4 Jun 2015 12:40:52 +0300
880 OK
881 OK
882 OK
883 OK
884 OK
885 OK
886 OK
887 !!!error - list index out of range Tue, 5 Dec 2017 16:20:36 +0300
887 OK
888 OK
889 !!!error - list index out of range Fri, 24 Nov 2017 21:58:45 +0300
889 OK
890 !!!error - list index out of range Thu, 30 Nov 2017 14:56:05 +0300
890 OK
891 !!!error - list index out of range Fri, 24 Nov 2017 13:23:36 +0300
891 OK
892 OK
893 OK
894 OK
895 OK
896 OK
897 OK
898 OK
899 !!!error - invalid literal for int() with base 10: '№' Tue, 14 Apr 2015 17:56:11 +0300
899 OK
900 OK
901 OK
902 OK
903 !!!error - invalid literal for int() with base 10: '№' Fri, 20 Feb 2015 10:59:08 +0300
903 OK
904 OK
905 OK
906 !!!error - invalid literal for int() with base 10: '№' Fri, 27 Feb 2015 08:12:48 +0300
906 !!!error - invalid literal for int() with base 10

1184 OK
1185 OK
1186 OK
1187 OK
1188 OK
1189 OK
1190 !!!error - invalid literal for int() with base 10: '24ae080' Thu, 30 Jul 2015 00:54:36 +0300
1190 !!!error - list index out of range Mon, 11 Sep 2017 09:55:36 +0300
1190 OK
1191 OK
1192 OK
1193 OK
1194 OK
1195 OK
1196 OK
1197 OK
1198 OK
1199 OK
1200 OK
1201 OK
1202 OK
1203 OK
1204 OK
1205 !!!error - expected string or bytes-like object Wed, 4 Jul 2018 15:37:31 +0300
1205 !!!error - list index out of range Thu, 14 Dec 2017 13:15:00 +0300
1205 OK
1206 OK
1207 OK
1208 OK
1209 OK
1210 OK
1211 OK
1212 OK
1213 OK
1214 OK
1215 OK
1216 OK
1217 OK
1218 OK
1219 OK
1220 OK
1221 OK
1222 !!!error - list index out of range Mon, 11 Dec 2017 21:42:10 +0300
1222 OK
1223 OK
1224 OK
1225 OK
1226 !!!error - invalid literal for int() with base 10: '5a65065' Wed, 22 Jul 2015 22:13:10 +0300
1226 OK
1227 OK
1228 OK
1229 OK
1230 OK
1231 !!!error - list index out of range Sun, 10 Dec 2017 12:53:12 +0300
1231 !!!error - invalid literal for int() with base 10: 

1546 OK
1547 OK
1548 OK
1549 OK
1550 OK
1551 !!!error - invalid literal for int() with base 10: 'Заказ' Sun, 29 Apr 2018 19:39:57 +0300
1551 OK
1552 OK
1553 OK
1554 OK
1555 OK
1556 OK
1557 OK
1558 OK
1559 OK
1560 OK
1561 OK
1562 OK
1563 OK
1564 OK
1565 OK
1566 OK
1567 OK
1568 !!!error - list index out of range Sat, 29 Dec 2018 10:33:15 +0300
1568 OK
1569 !!!error - invalid literal for int() with base 10: '№' Wed, 4 Mar 2015 16:49:34 +0300
1569 OK
1570 OK
1571 OK
1572 OK
1573 OK
1574 OK
1575 OK
1576 OK
1577 !!!error - list index out of range Wed, 6 Dec 2017 15:04:52 +0300
1577 OK
1578 OK
1579 OK
1580 OK
1581 OK
1582 OK
1583 OK
1584 OK
1585 OK
1586 OK
1587 OK
1588 OK
1589 OK
1590 !!!error - invalid literal for int() with base 10: '№' Tue, 30 Jun 2015 12:27:58 +0300
1590 OK
1591 !!!error - invalid literal for int() with base 10: '№' Sat, 7 Mar 2015 11:41:28 +0300
1591 OK
1592 OK
1593 OK
1594 OK
1595 OK
1596 OK
1597 !!!error - list index out of range Fri, 8 Dec 2017 12:45:38 +0300
1597 OK


...

3975 OK
3976 OK
3977 OK
3978 OK
3979 OK
3980 OK
3981 OK
3982 OK
3983 OK
3984 OK
3985 OK
3986 OK
3987 !!!error - expected string or bytes-like object Mon, 12 Dec 2016 21:10:02 +0300
3987 OK
3988 OK
3989 OK
3990 !!!error - invalid literal for int() with base 10: '№' Thu, 16 Apr 2015 23:22:57 +0300
3990 OK
3991 OK
3992 OK
3993 OK
3994 OK
3995 OK
3996 OK
3997 OK
3998 OK
3999 OK
4000 OK
commit
4001 OK
4002 OK
4003 OK
4004 OK
4005 OK
4006 OK
4007 OK
4008 OK
4009 OK
4010 OK
4011 !!!error - expected string or bytes-like object Fri, 29 Sep 2017 12:48:41 +0300
4011 !!!error - list index out of range Thu, 19 Apr 2018 17:56:19 +0300
4011 OK
4012 OK
4013 OK
4014 OK
4015 OK
4016 OK
4017 !!!error - list index out of range Wed, 13 Dec 2017 12:53:15 +0300
4017 OK
4018 OK
4019 OK
4020 !!!error - list index out of range Tue, 12 Dec 2017 19:25:25 +0300
4020 !!!error - invalid literal for int() with base 10: '№' Fri, 27 Feb 2015 10:54:29 +0300
4020 OK
4021 !!!error - expected string or bytes-like object Th

4363 OK
4364 OK
4365 OK
4366 OK
4367 OK
4368 OK
4369 OK
4370 OK
4371 OK
4372 OK
4373 !!!error - invalid literal for int() with base 10: '№' Tue, 7 Apr 2015 00:25:33 +0300
4373 OK
4374 OK
4375 !!!error - list index out of range Wed, 13 Dec 2017 16:17:50 +0300
4375 !!!error - list index out of range Sun, 26 Nov 2017 16:51:06 +0300
4375 !!!error - expected string or bytes-like object Thu, 19 Feb 2015 13:53:30 +0003
4375 OK
4376 OK
4377 OK
4378 OK
4379 OK
4380 !!!error - list index out of range Tue, 9 Oct 2018 12:13:31 +0300
4380 OK
4381 OK
4382 OK
4383 OK
4384 !!!error - list index out of range Tue, 12 Dec 2017 12:31:35 +0300
4384 OK
4385 OK
4386 OK
4387 OK
4388 OK
4389 !!!error - list index out of range Mon, 4 Dec 2017 12:56:12 +0300
4389 !!!error - invalid literal for int() with base 10: '№' Fri, 6 Mar 2015 12:23:48 +0300
4389 OK
4390 OK
4391 !!!error - invalid literal for int() with base 10: '3616-1' Wed, 12 Jul 2017 19:25:19 +0300
4391 OK
4392 OK
4393 OK
4394 OK
4395 OK
4396 !!!error 

4720 OK
4721 OK
4722 OK
4723 OK
4724 OK
4725 OK
4726 OK
4727 OK
4728 OK
4729 !!!error - list index out of range Fri, 1 Dec 2017 13:30:46 +0300
4729 !!!error - list index out of range Mon, 27 Mar 2017 12:50:26 +0300
4729 OK
4730 OK
4731 OK
4732 OK
4733 OK
4734 OK
4735 OK
4736 !!!error - list index out of range Thu, 7 Dec 2017 14:11:35 +0300
4736 OK
4737 OK
4738 OK
4739 !!!error - list index out of range Wed, 22 Nov 2017 12:10:02 +0300
4739 OK
4740 !!!error - list index out of range Fri, 2 Dec 2016 09:23:38 +0300
4740 !!!error - list index out of range Mon, 30 Jul 2018 15:29:12 +0300
4740 OK
4741 OK
4742 OK
4743 OK
4744 OK
4745 !!!error - list index out of range Wed, 29 Nov 2017 22:22:58 +0300
4745 OK
4746 !!!error - list index out of range Sat, 9 Dec 2017 20:07:05 +0300
4746 OK
4747 OK
4748 OK
4749 OK
4750 OK
commit
4751 OK
4752 OK
4753 OK
4754 OK
4755 OK
4756 OK
4757 OK
4758 OK
4759 OK
4760 OK
4761 !!!error - invalid literal for int() with base 10: '№' Thu, 16 Apr 2015 09:33:14 +0300
4

5113 OK
5114 !!!error - list index out of range Thu, 28 Mar 2019 11:31:28 +0300
5114 OK
5115 OK
5116 OK
5117 OK
5118 OK
5119 OK
5120 OK
5121 OK
5122 OK
5123 OK
5124 OK
5125 OK
5126 OK
5127 OK
5128 OK
5129 OK
5130 OK
5131 OK
5132 OK
5133 OK
5134 OK
5135 OK
5136 OK
5137 OK
5138 OK
5139 OK
5140 OK
5141 OK
5142 OK
5143 !!!error - list index out of range Sun, 18 Dec 2016 22:01:26 +0300
5143 OK
5144 OK
5145 OK
5146 OK
5147 OK
5148 OK
5149 OK
5150 OK
5151 OK
5152 OK
5153 OK
5154 OK
5155 OK
5156 OK
5157 OK
5158 OK
5159 OK
5160 OK
5161 OK
5162 OK
5163 OK
5164 OK
5165 OK
5166 OK
5167 OK
5168 OK
5169 OK
5170 OK
5171 OK
5172 OK
5173 OK
5174 !!!error - list index out of range Fri, 1 Dec 2017 14:09:08 +0300
5174 OK
5175 OK
5176 OK
5177 OK
5178 OK
5179 OK
5180 OK
5181 OK
5182 OK
5183 OK
5184 OK
5185 OK
5186 OK
5187 OK
5188 !!!error - list index out of range Wed, 13 Dec 2017 14:06:44 +0300
5188 !!!error - invalid literal for int() with base 10: '808a039' Thu, 16 Jul 2015 17:10:38 +0300
5188 !!!error -

5546 OK
5547 OK
5548 OK
5549 OK
5550 OK
5551 OK
5552 OK
5553 OK
5554 OK
5555 OK
5556 OK
5557 OK
5558 OK
5559 OK
5560 !!!error - invalid literal for int() with base 10: '№' Mon, 16 Feb 2015 12:27:41 +0300
5560 OK
5561 OK
5562 OK
5563 OK
5564 OK
5565 OK
5566 OK
5567 OK
5568 OK
5569 OK
5570 OK
5571 OK
5572 OK
5573 OK
5574 OK
5575 OK
5576 OK
5577 OK
5578 OK
5579 OK
5580 OK
5581 OK
5582 OK
5583 OK
5584 OK
5585 OK
5586 OK
5587 OK
5588 OK
5589 OK
5590 OK
5591 OK
5592 OK
5593 OK
5594 OK
5595 !!!error - list index out of range Thu, 7 Dec 2017 15:10:12 +0300
5595 !!!error - invalid literal for int() with base 10: '№' Mon, 29 Jun 2015 14:29:14 +0300
5595 !!!error - invalid literal for int() with base 10: '3a2e052' Fri, 17 Jul 2015 10:52:44 +0300
5595 OK
5596 OK
5597 OK
5598 OK
5599 OK
5600 OK
5601 OK
5602 OK
5603 OK
5604 OK
5605 OK
5606 OK
5607 OK
5608 OK
5609 OK
5610 OK
5611 OK
5612 OK
5613 OK
commit


In [ ]:

import dbstruct
engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)


#dbstruct.Base.metadata.drop_all(engine)
#dbstruct.Base.metadata.create_all(engine)

engine.dispose()

In [ ]:
engine.dispose()